In [ ]:
#-------------------------
# Libs
#-------------------------
 
# External libs
# %matplotlib qt         Fuck this line right here.
import pymysql.cursors
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.neural_network import MLPRegressor as mlp
sys.path.append("/home/omalleyian/Documents/energy_market_project/scripts")
from ercot_data_interface import ercot_data_interface
from ARIMA import ARIMA

In [ ]:
#-------------------------
# SQL environments
#-------------------------
 
HOST = "localhost"
USER = "root" 
PASSWORD = "Is79t5Is79t5"
DB = "ercot_data"

#-------------------------
# Functions
#-------------------------
"""
Make a connection to MySQL
Execute the MySQL query and return the resutls
"""

def execute_dict_query(query):
    connection = pymysql.connect(host=HOST, 
                                 user=USER, 
                                 password=PASSWORD, 
                                 db=DB, 
                                 port=3306,
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(query)
            result = cursor.fetchall()
            return result
    finally:
        connection.close()


In [ ]:
def mape(actual, forcast):
    x = (np.asarray(actual)).flatten()
    y = (np.asarray(forcast)).flatten()
    return np.mean(np.square(np.abs((x - y)) / x))

def mae(actual, forcast):
    x = (np.asarray(actual)).flatten()
    y = (np.asarray(forcast)).flatten()
    return np.mean(np.abs((x - y)))

def mase(actual, forcast):
    x = (np.asarray(actual)).flatten()
    y = (np.asarray(forcast)).flatten()
    return np.mean(np.abs(x - y) / np.mean(np.abs(x[1:]-x[:-1])))
    
def hit_rate(actual, forcast):
    x = (np.asarray(actual)).flatten()
    y = (np.asarray(forcast)).flatten()
    f_t = (x[1:] - x[:-1]) * (y[1:] - y[:-1])
    # k is the subset of f_t where k[i] = f_t[i] iff f_t[i] > 0
    k = [i for i in f_t if i > 0]
    return np.mean(np.abs(k))

def hit_rate2(actual, forcast):
    x = (np.asarray(actual)).flatten()
    y = (np.asarray(forcast)).flatten()
    f_t = (x[1:] - x[:-1]) * (y[1:] - y[:-1])  
    # k is the subset of y where k[i] = y[i] iff f_t[i] > 0
    k = [y[i+1] for i in range(f_t.shape[0]) if f_t[i] > 0]
    return np.mean(np.abs(k))

In [ ]:
# Query data sorted by date and hour. SLOW!
result_dict = execute_dict_query('select * from DAM_LMP0  \
                                where delivery_date < "2016-12-31" \
                                and delivery_date > "2011-01-01" \
                                order by delivery_date, hour_ending')

In [ ]:
#Extract some data
prices1 = [i['n0001VICTOR'] for i in result_dict]
prices2 = [i['n0001'] for i in result_dict]
hours = [i['hour_ending'].total_seconds() for i in result_dict]
dates = [i['delivery_date'] for i in result_dict]
plt.plot(prices1)
plt.plot(prices2)
# plt.show()

In [ ]:
# Percent Change
k = 24
p_t = np.array(prices1[k:])
p_tk = np.array(prices1[:-k])
v = np.log10(p_t) - np.log10(p_tk)
plt.plot(v)
# plt.show()

# MAPE = mean( (actual - forcast) / actual)

MAPE = np.mean(np.square(p_t-p_tk)/p_t)

In [ ]:
# Basic data plot
price1 = [float(i['AMNCOWD_8']) for i in result_dict]
price2 = [float(i['AZLES_K']) for i in result_dict]
price3 = [float(i['n0001']) for i in result_dict]

plt.plot(price1, label='1')
plt.plot(price2, label='2')
plt.plot(price3, label='2')
plt.legend()
# plt.show()


In [ ]:
# Histogram
plt.hist(price1, 500)
plt.show()

In [ ]:
# Autocorrelation
plt.acorr(price1, maxlags=72)
plt.show()

In [ ]:
np.mean(price1)

In [ ]:
#extend this to get prices at different lags e.g. k-1, k-2, k-3
max_lag = 168
p_t = prices1[max_lag:]
X = []
for k in np.arange(0, max_lag, 24):
    t_k = np.array(prices1[(max_lag-k):-k], dtype='float')
    X.append(t_k)
X = np.swapaxes(np.array(X[1:]), 0, 1)
Y = [float(i) for i in p_t]
# use sklearn.LinearRegression to fit (X, Y)

regr = linear_model.LinearRegression()
regr.fit(X,Y)

print regr.predict(X)
plt.plot(X)
# plt.show()


In [ ]:
# Moving Average Model
# Given a list X, time t, and m number of observations
# Return the value of Y at time t of the previous m elements in X.

def moving_average(X, t, m):
    s = sum(X[(t-m):t])
    avg = s/m
    return avg

print moving_average(prices1, 24, 24)

In [ ]:
ercot = ercot_data_interface(password="Is79t5Is79t5")
nodes_crr = ercot.get_CRR_nodes()
nodes_all = ercot.all_nodes
nodes_source = ercot.get_sources_sinks()
df_2011 = ercot.query_prices(nodes_all[0], "2011-01-01","2011-12-31")
df_2012 = ercot.query_prices(nodes_all[0], "2012-01-01","2012-12-31")
matrix_2011 = df_2011.as_matrix()
matrix_2012 = df_2012.as_matrix()

In [ ]:
arima = ARIMA(p = 2, d = 0, q = 0, seasonal = 24)
arima.fit(matrix_2011)
arima.plot_predicted_vs_actual(matrix_2012)
print arima.mae(matrix_2012)

In [ ]:
# Arima Forcasting with variable horizon

def arima_forcast(train, test, horizon, p, d, q, seasonal):
    # First split data into subsets offset by 24, and fit ARIMA models to these subsets.
    hours_train = []
    hours_test = []
    arima_models = []
    loss = p + q + seasonal - 1

    for i in range(horizon):
        ind_train = np.arange(i,train.shape[0],horizon)
        ind_test = np.arange(i,test.shape[0],horizon)

        hours_train.append(train[ind_train])
        hours_test.append(test[ind_test])

        arima = ARIMA(p, d, q, seasonal)
        arima.fit(train[ind_train])
        arima_models.append(arima)

    # Make predictions with the ARIMA models
    # Merge the predictions into a single array to be ploted against actual data
    merged_prediction = np.zeros(horizon * (test.shape[0] / horizon - loss))
    merged_actual = np.zeros(horizon * (test.shape[0] / horizon - loss))
    
    for i in range(len(arima_models)):
        prediction, actual = arima_models[i].predict(hours_test[i])
        prediction = prediction.squeeze().tolist()
        actual = actual.squeeze().tolist()

        if len(prediction) < (len(test) / horizon) - loss:
            prediction.extend(np.zeros((len(test) / horizon - loss) - len(prediction)))
            actual.extend(np.zeros((len(test) / horizon - loss) - len(actual)))

        for j in range(len(prediction)):
            front = [0] * i
            back = [0] * (horizon-1-i)
            index = j * horizon

            prediction[index:index] = front
            actual[index:index] = front

            prediction[index+i+1:index+i+1] = back
            actual[index+i+1:index+i+1] = back

        merged_prediction += prediction
        merged_actual += actual
        
    return merged_prediction, merged_actual

In [ ]:
merged_prediction, merged_actual = \
    arima_forcast(matrix_2011, matrix_2012, 24, 2, 0, 1, 1)
plt.plot(merged_actual, label='actual')
plt.plot(merged_prediction, label='prediction', alpha=0.5)
plt.legend()
plt.show()

print "Merged MAE: " + str(mae(merged_actual, merged_prediction))

In [ ]:
# Calculate mape for various models on the first 5 nodes
for i in range(5):
    func = hit_rate2
    print "MAE Calculation on Node" + str(i) + ": " + str(nodes_all[i])
    # Load training and testing data from a node
    df_train = ercot.query_prices(nodes_all[i], "2011-01-01","2015-05-23")
    df_test = ercot.query_prices(nodes_all[i], "2015-05-23","2016-05-23")
    matrix_train = df_train.as_matrix()
    matrix_test = df_test.as_matrix()
    
    # Preform mean model calculations on data
    m = np.mean(matrix_train)
    m_matrix = [m] * len(matrix_test)
    print func(matrix_test, m_matrix)
    
    # Perform random walk model calculations on data
    walk = matrix_test[:-1]
    walk_test = matrix_test[1:]
    print func(walk_test, walk)
    
    # ARIMA(2,0,1,1)
    arima = ARIMA(p = 2, d = 0, q = 1, seasonal = 1)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)
    
    # ARIMA(2,0,2,1)
    arima = ARIMA(p = 2, d = 0, q = 2, seasonal = 1)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)
    
    # ARIMA(2,0,3,1)
    arima = ARIMA(p = 2, d = 0, q = 3, seasonal = 1)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)
    
    # ARIMA(2,0,1,24)
    arima = ARIMA(p = 2, d = 0, q = 1, seasonal = 24)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)    
    
    # ARIMA(2,0,2,24)
    arima = ARIMA(p = 2, d = 0, q = 2, seasonal = 24)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)
    
    # ARIMA(2,0,3,24)
    arima = ARIMA(p = 2, d = 0, q = 3, seasonal = 24)
    arima.fit(matrix_train)
    actual, forcast = arima.predict(matrix_test)
    print func(actual, forcast)
    
    print "\n"

In [ ]:
df_train = ercot.query_prices(nodes_all[0], "2011-01-01","2015-05-23")
df_test = ercot.query_prices(nodes_all[0], "2015-05-23","2016-05-23")
matrix_train = df_train.as_matrix()
matrix_test = df_test.as_matrix()

In [ ]:
arima = ARIMA(p = 2, d = 0, q = 1, seasonal = 1)
arima.fit(matrix_train)
print str(arima.mae(matrix_test))
arima = ARIMA(p = 2, d = 0, q = 2, seasonal = 1)
arima.fit(matrix_train)
print str(arima.mae(matrix_test))
arima = ARIMA(p = 2, d = 0, q = 3, seasonal = 1)
arima.fit(matrix_train)
print str(arima.mae(matrix_test))
print "\n"

arima = ARIMA(p = 2, d = 0, q = 3, seasonal = 20)
arima.fit(matrix_train)
print str(arima.mae(matrix_test))

merged_prediction, merged_actual = \
    arima_forcast(matrix_train, matrix_test, 24, 2, 0, 1, 1)
plt.plot(merged_actual, label='actual')
plt.plot(merged_prediction, label='prediction', alpha=0.5)
plt.legend()
plt.show()

print "Merged MAE: " + str(mae(merged_actual, merged_prediction))